In [9]:
import json
from collections import defaultdict
import re

In [47]:

recipes = []

graph = defaultdict(list)
replacements = []

ingredient_bag = defaultdict(int)


def parse_ingredient(line):

    line = line.lstrip()
    idx = 0
    # devemos tentar aqui separar as unidades ou simplesmente unidades + elementos são o proprio elemento?
    #while (line[idx].isdigit() or line[idx] == '/' or line[idx] == ' '):
    #    idx+=1
    
    #quantity = line [0 : idx].lstrip()
    
    ingredients = line[idx:].lower()

    ingredients = ingredients.split(' or ')

    for i in range(len(ingredients)):
        ingredients[i] = ingredients[i].lstrip()

    return quantity, ingredients

with open('./data.json') as json_file:
    data = json.load(json_file)
    for p in data:
        recipes.append(p)
        for i in range (len(p['ingredients']) - 1):
            for j in range( i + 1, len(p['ingredients'])):

                quantity, ingredients = parse_ingredient(p['ingredients'][i])
                quantity2, ingredients2 = parse_ingredient(p['ingredients'][j]) 

                if len(ingredients) > 1:
                    replacements.append(ingredients)

                if len(ingredients2) > 1:
                    replacements.append(ingredients2)

                for ing in ingredients:
                    graph[ing] += ingredients2
                    ingredient_bag[ing]+=1

                for ing in ingredients2:
                    graph[ing] += ingredients
                    ingredient_bag[ing]+=1
                


for k, v in ingredient_bag.items():
    print("{}->{}".format(k, v))
#for n in graph:
#    print("{} -> {}".format(n , len(graph[n])))

1 tablespoon orange zest (from one orange)->26
10 large dried turkish->26
caliymirna figs (the light brown ones)->26
1 teaspoon freshly ground nutmeg->26
1 teaspoon ground cinnamon->94
1 cup whole milk->26
1 1/2 cups all purpose flour->26
1 teaspoon kosher salt->70
3/4 cup unsalted butter, softened->26
3/4 cup powdered sugar->26
1 large egg->144
1 1/2 cups powdered sugar->26
1/2 cup unsalted butter, softened->46
2 tablespoons brandy->26
1/2 teaspoon vanilla extract->112
1/2 cup unsalted butter, at room temperature->16
1/2 cup granulated sugar->16
1/4 cup light brown sugar->34
2 teaspoons vanilla extract->54
1 1/2 cups all-purpose unbleached flour (i use king arthur)->16
1/2 teaspoon sea salt->16
1/4 teaspoon baking soda->60
1 cup turbinado,->36
coarse, sugar->16
2 3/4 cups all-purpose flour->16
1 teaspoon baking soda->266
1/2 teaspoon baking powder->116
1 cup butter, softened->32
1 1/2 cups white sugar->16
1 egg->196
1 teaspoon vanilla extract->278
1/4 cup grated coconut, plus more for

In [74]:
from queue import PriorityQueue

def calculate_distance(graph, ingredient):
    pq = PriorityQueue()
    dist = defaultdict(int)

    pq.put((0, ingredient ))
    dist[ingredient] = 0
    visited = set()

    while not pq.empty():
        u = pq.get()[1]
        for i in graph[u]:
            if i in visited:
                continue
            v = i
            visited.add(i)
            if v not in dist or dist[v] > dist[u] + 1:
                dist[v] = dist[u] + 1
                pq.put((dist[v], v))
    
    return dist



def calculate_distances(graph):   
    dist = {}
    for ingredient in graph:
        dist[ingredient] = calculate_distance(graph, ingredient)

    return dist


In [91]:
dist = calculate_distances(graph) # act as a cache


In [108]:
#print(dist.keys())

In [107]:

def fitness(individual):
    counter = 0
    # the higher the worst
    pq = PriorityQueue()
    visited = set()
    for i in range(len(individual['ingredients']) - 1):
        for j in range(i + 1 , len(individual['ingredients'])):
            if i in visited and j in visited:
                continue
            pq.put(dist[individual['ingredients'][i]][individual['ingredients'][j]])

    while not pq.empty():
        u = pq.get()
        counter += u
    return counter

        

print(fitness(recipes[4])) 
print(fitness(recipes[7]))


36
78
